In [1]:
import requests, json
import pyecharts
import pandas as pd
from pyecharts.charts import Pie
from pyecharts import options as opts

In [2]:
# 获取新冠肺炎最新的全球数据
resp = requests.get("https://lab.isaaclin.cn/nCoV/api/area?latest=1")
data_json=resp.json()
data_global=data_json.get("results")
df_global = pd.read_json(json.dumps(data_global),orient='records')

In [3]:
# 查看数据
df_global.head()

,cities,comment,confirmedCount,continentName,countryName,curedCount,currentConfirmedCount,deadCount,locationId,provinceName,provinceShortName,suspectedCount,updateTime
0,None,,509,亚洲,新加坡,152,355,2,952009,新加坡,新加坡,0,1584971012451
1,None,,1450,欧洲,丹麦,1,1436,13,962001,丹麦,丹麦,0,1584970149379
2,None,,1089,亚洲,日本,285,763,41,951002,日本,日本,0,1584970149379
3,None,,2060,欧洲,葡萄牙,14,2023,23,965010,葡萄牙,葡萄牙,0,1584969155012
4,None,,3611,欧洲,奥地利,9,3586,16,963001,奥地利,奥地利,0,1584968822828


In [4]:
# 只取关心的数据列
selected_cloumns = ['countryName','provinceShortName','provinceName','confirmedCount','currentConfirmedCount','deadCount','curedCount','updateTime']
# 过滤出确诊人数过千的海外国家的数据并按确诊人数倒序排列
df_selected_oversea=df_global[df_global.countryName!='中国'][selected_cloumns]
df_sorted_oversea=df_selected_oversea[df_selected_oversea.confirmedCount>1000].sort_values(by='confirmedCount', ascending=False)

In [5]:
# 查看过滤排序后的数据
df_sorted_oversea.head()

,countryName,provinceShortName,provinceName,confirmedCount,currentConfirmedCount,deadCount,curedCount,updateTime
170,意大利,意大利,意大利,59514,47014,5476,7024,1584963423867
6,美国,美国,美国,35227,34580,471,176,1584968286434
13,西班牙,西班牙,西班牙,33089,27552,2182,3355,1584964629818
44,德国,德国,德国,26220,25843,111,266,1584963423867
158,伊朗,伊朗,伊朗,23049,13324,1812,7913,1584963423867


In [6]:
# 定义绘制南丁格尔玫瑰图的函数
def plot_rose(item_list, data_list, title_string):
    ziped_item_data_list = [list(z) for z in zip(item_list, data_list)]

    rose_diagram = Pie(init_opts=opts.InitOpts(width='650px', height='650px'))
    rose_diagram.add("", ziped_item_data_list, radius=["40%", "130%"], center=["50%", "70%"], rosetype="area")
    rose_diagram.set_global_opts(title_opts=opts.TitleOpts(title=title_string),
                                 legend_opts=opts.LegendOpts(is_show=False),
                                 toolbox_opts=opts.ToolboxOpts()
                                )
    label_opts = opts.LabelOpts(is_show=True, position="inside", font_size=12, formatter="{b}:{c}人")
    rose_diagram.set_series_opts(label_opts=label_opts)
    rose_diagram.render('{}.html'.format(title_string))
    return rose_diagram

In [7]:
# 准备绘图使用的数据，仅取海外确诊过千国家的数据
item_oversea_list = df_sorted_oversea['countryName'].values.tolist()
data_oversea_list = df_sorted_oversea['confirmedCount'].values.tolist()
title_string = "海外各国确诊人数（仅过千的国家）"

In [8]:
# 绘制玫瑰图
rose2 = plot_rose(item_oversea_list, data_oversea_list, title_string)

In [9]:
# 将绘制的图渲染在juypter notebook中
rose2.render_notebook()